In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold


# LOAD
X_train = pd.read_csv('flight_delays_train.csv')
X_test = pd.read_csv('flight_delays_test.csv')

y_train = X_train.iloc[:, -1].map({'Y': 1, 'N': 0})
X_train = X_train.iloc[:, :-1]



# CATEGORIAL PREPROCESS
train_month = X_train['Month'].astype('category')
test_month = X_test['Month'].astype('category', categories=list(train_month.cat.categories))
train_month = pd.get_dummies(train_month)
test_month = pd.get_dummies(test_month)

train_dayofmonth = X_train['DayofMonth'].astype('category')
test_dayofmonth = X_test['DayofMonth'].astype('category', categories=list(train_dayofmonth.cat.categories))
train_dayofmonth = pd.get_dummies(train_dayofmonth)
test_dayofmonth = pd.get_dummies(test_dayofmonth)

train_dayofweek = X_train['DayOfWeek'].astype('category')
test_dayofweek = X_test['DayOfWeek'].astype('category', categories=list(train_dayofweek.cat.categories))
train_dayofweek = pd.get_dummies(train_dayofweek)
test_dayofweek = pd.get_dummies(test_dayofweek)

train_uniquecarrier = X_train['UniqueCarrier'].astype('category')
test_uniquecarrier = X_test['UniqueCarrier'].astype('category', categories=list(train_uniquecarrier.cat.categories))
train_uniquecarrier = pd.get_dummies(train_uniquecarrier)
test_uniquecarrier = pd.get_dummies(test_uniquecarrier)

train_origin = X_train['Origin'].astype('category')
test_origin = X_test['Origin'].astype('category', categories=list(train_origin.cat.categories))
train_origin = pd.get_dummies(train_origin)
test_origin = pd.get_dummies(test_origin)

train_dest = X_train['Dest'].astype('category')
test_dest = X_test['Dest'].astype('category', categories=list(train_dest.cat.categories))
train_dest = pd.get_dummies(train_dest)
test_dest = pd.get_dummies(test_dest)

train_route = (X_train['Dest'] + ' - ' + X_train['Origin']).astype('category')
test_route = (X_test['Dest'] + ' - ' + X_test['Origin']).astype('category', categories=list(train_route.cat.categories))
train_route = pd.get_dummies(train_dest)
test_route = pd.get_dummies(test_dest)

X_train_preproc = np.hstack((train_month, train_dayofmonth, train_dayofweek, X_train['DepTime'].reshape(-1, 1), train_uniquecarrier, train_origin, train_dest, X_train['Distance'].reshape(-1, 1),train_route))
X_test_preproc = np.hstack((test_month, test_dayofmonth, test_dayofweek, X_test['DepTime'].reshape(-1, 1), test_uniquecarrier, test_origin, test_dest, X_test['Distance'].reshape(-1, 1),test_route))

#print('oversampling')
#from imblearn.over_sampling import SMOTE
#smote = SMOTE(kind='svm', random_state=17)
#X_train_preproc, y_train = smote.fit_sample(X_train_preproc, y_train)
#from sklearn.utils import shuffle
#X_train_preproc, y_train = shuffle(X_train_preproc, y_train, random_state=17)









# CROSS VALIDATION AND HOLDOUT
cv_holdout_splitter = int(0.8 * X_train_preproc.shape[0])

X_train_preproc_cv = X_train_preproc[:cv_holdout_splitter, :]
y_train_cv = y_train[:cv_holdout_splitter]

X_train_preproc_holdout = X_train_preproc[cv_holdout_splitter:, :]
y_train_holdout = y_train[cv_holdout_splitter:]

print('start fitting')
gbm = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)


cross_val_score = np.mean(cross_val_score(gbm, X_train_preproc_cv, y_train_cv, cv=skf, scoring='roc_auc'))

gbm = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary')
gbm.fit(X_train_preproc_cv, y_train_cv)
holdout_predictions = gbm.predict_proba(X_train_preproc_holdout)[:, 1]
holdout_score = roc_auc_score(y_train_holdout, holdout_predictions)

print(cross_val_score, holdout_score)



 #SUBMISSION
gbm = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary')
gbm.fit(X_train_preproc, y_train)

test_predictions = gbm.predict_proba(X_test_preproc)[:, 1]
pd.Series(test_predictions, name='dep_delayed_15min').to_csv('gbm2.csv', index_label='id', header=True)  #0.7306





start fitting
0.7374922274377456 0.7470460239796421


In [3]:
from catboost import CatBoostClassifier
# Initialize data
X_train['route'] = X_train.Origin + ' - ' + X_train.Dest
X_test['route'] = X_test.Origin + ' - ' + X_test.Dest
cat_features = np.where(X_train.dtypes == 'object')[0].tolist()
train_data = X_train.values[:cv_holdout_splitter, :]
train_labels = y_train.values[:cv_holdout_splitter]
test_data = X_train.values[cv_holdout_splitter:]
# Initialize CatBoostClassifier
model = CatBoostClassifier(random_seed=17)
# Fit model
model.fit(train_data, train_labels, cat_features)
# Get predicted classes
preds_class = model.predict(test_data)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(test_data)
# Get predicted RawFormulaVal
preds_raw = model.predict(test_data, prediction_type='RawFormulaVal')

holdout_score = roc_auc_score(y_train.values[cv_holdout_splitter:], preds_proba[:, 1])


#submissiom

model.fit(X_train, y_train, cat_features)
test_predictions2 = model.predict_proba(X_test)[:, 1]
pd.Series(test_predictions2*0.4+test_predictions*0.6, name='dep_delayed_15min').to_csv('union.csv', index_label='id', header=True)  #0.7306



0:	learn: 0.6759347	total: 157ms	remaining: 2m 37s
1:	learn: 0.6600665	total: 225ms	remaining: 1m 52s
2:	learn: 0.6456125	total: 281ms	remaining: 1m 33s
3:	learn: 0.6320490	total: 348ms	remaining: 1m 26s
4:	learn: 0.6196542	total: 420ms	remaining: 1m 23s
5:	learn: 0.6081353	total: 487ms	remaining: 1m 20s
6:	learn: 0.5971566	total: 546ms	remaining: 1m 17s
7:	learn: 0.5869098	total: 606ms	remaining: 1m 15s
8:	learn: 0.5774262	total: 669ms	remaining: 1m 13s
9:	learn: 0.5683523	total: 739ms	remaining: 1m 13s
10:	learn: 0.5599254	total: 796ms	remaining: 1m 11s
11:	learn: 0.5520762	total: 866ms	remaining: 1m 11s
12:	learn: 0.5447892	total: 926ms	remaining: 1m 10s
13:	learn: 0.5384382	total: 1.01s	remaining: 1m 11s
14:	learn: 0.5324005	total: 1.1s	remaining: 1m 12s
15:	learn: 0.5268815	total: 1.17s	remaining: 1m 12s
16:	learn: 0.5210492	total: 1.24s	remaining: 1m 11s
17:	learn: 0.5162212	total: 1.3s	remaining: 1m 11s
18:	learn: 0.5116854	total: 1.37s	remaining: 1m 10s
19:	learn: 0.5074191	tot

162:	learn: 0.4251157	total: 11.5s	remaining: 59.1s
163:	learn: 0.4250264	total: 11.6s	remaining: 59s
164:	learn: 0.4249285	total: 11.6s	remaining: 58.9s
165:	learn: 0.4248796	total: 11.7s	remaining: 58.7s
166:	learn: 0.4247804	total: 11.7s	remaining: 58.6s
167:	learn: 0.4247043	total: 11.8s	remaining: 58.5s
168:	learn: 0.4246187	total: 11.9s	remaining: 58.3s
169:	learn: 0.4245185	total: 11.9s	remaining: 58.2s
170:	learn: 0.4244670	total: 12s	remaining: 58.2s
171:	learn: 0.4244005	total: 12.1s	remaining: 58.1s
172:	learn: 0.4243293	total: 12.1s	remaining: 58s
173:	learn: 0.4242809	total: 12.2s	remaining: 57.9s
174:	learn: 0.4242231	total: 12.3s	remaining: 57.8s
175:	learn: 0.4241487	total: 12.3s	remaining: 57.7s
176:	learn: 0.4240863	total: 12.4s	remaining: 57.7s
177:	learn: 0.4240187	total: 12.5s	remaining: 57.6s
178:	learn: 0.4239626	total: 12.5s	remaining: 57.5s
179:	learn: 0.4238812	total: 12.6s	remaining: 57.5s
180:	learn: 0.4237719	total: 12.7s	remaining: 57.4s
181:	learn: 0.4237

322:	learn: 0.4171666	total: 22.3s	remaining: 46.8s
323:	learn: 0.4171490	total: 22.4s	remaining: 46.8s
324:	learn: 0.4171240	total: 22.5s	remaining: 46.7s
325:	learn: 0.4170889	total: 22.5s	remaining: 46.6s
326:	learn: 0.4170721	total: 22.6s	remaining: 46.5s
327:	learn: 0.4170490	total: 22.7s	remaining: 46.5s
328:	learn: 0.4170044	total: 22.7s	remaining: 46.4s
329:	learn: 0.4169804	total: 22.8s	remaining: 46.4s
330:	learn: 0.4169503	total: 22.9s	remaining: 46.3s
331:	learn: 0.4169081	total: 23s	remaining: 46.2s
332:	learn: 0.4168542	total: 23s	remaining: 46.1s
333:	learn: 0.4168349	total: 23.1s	remaining: 46.1s
334:	learn: 0.4167924	total: 23.2s	remaining: 46s
335:	learn: 0.4167629	total: 23.2s	remaining: 45.9s
336:	learn: 0.4167321	total: 23.3s	remaining: 45.9s
337:	learn: 0.4166949	total: 23.4s	remaining: 45.9s
338:	learn: 0.4166642	total: 23.5s	remaining: 45.8s
339:	learn: 0.4166163	total: 23.6s	remaining: 45.8s
340:	learn: 0.4165937	total: 23.7s	remaining: 45.7s
341:	learn: 0.4165

482:	learn: 0.4131721	total: 33.5s	remaining: 35.9s
483:	learn: 0.4131599	total: 33.6s	remaining: 35.8s
484:	learn: 0.4131452	total: 33.7s	remaining: 35.8s
485:	learn: 0.4131256	total: 33.7s	remaining: 35.7s
486:	learn: 0.4131059	total: 33.8s	remaining: 35.6s
487:	learn: 0.4130954	total: 33.9s	remaining: 35.5s
488:	learn: 0.4130811	total: 33.9s	remaining: 35.5s
489:	learn: 0.4130635	total: 34s	remaining: 35.4s
490:	learn: 0.4130486	total: 34.1s	remaining: 35.3s
491:	learn: 0.4130109	total: 34.1s	remaining: 35.2s
492:	learn: 0.4129670	total: 34.2s	remaining: 35.2s
493:	learn: 0.4129464	total: 34.2s	remaining: 35.1s
494:	learn: 0.4128947	total: 34.3s	remaining: 35s
495:	learn: 0.4128579	total: 34.3s	remaining: 34.9s
496:	learn: 0.4128442	total: 34.4s	remaining: 34.8s
497:	learn: 0.4128312	total: 34.5s	remaining: 34.7s
498:	learn: 0.4128138	total: 34.5s	remaining: 34.7s
499:	learn: 0.4127906	total: 34.6s	remaining: 34.6s
500:	learn: 0.4127712	total: 34.7s	remaining: 34.5s
501:	learn: 0.41

642:	learn: 0.4099877	total: 44.6s	remaining: 24.8s
643:	learn: 0.4099565	total: 44.6s	remaining: 24.7s
644:	learn: 0.4099350	total: 44.7s	remaining: 24.6s
645:	learn: 0.4099033	total: 44.8s	remaining: 24.5s
646:	learn: 0.4098792	total: 44.9s	remaining: 24.5s
647:	learn: 0.4098437	total: 44.9s	remaining: 24.4s
648:	learn: 0.4098220	total: 45s	remaining: 24.3s
649:	learn: 0.4097746	total: 45s	remaining: 24.2s
650:	learn: 0.4097535	total: 45.1s	remaining: 24.2s
651:	learn: 0.4097318	total: 45.2s	remaining: 24.1s
652:	learn: 0.4097159	total: 45.2s	remaining: 24s
653:	learn: 0.4097064	total: 45.3s	remaining: 24s
654:	learn: 0.4096816	total: 45.4s	remaining: 23.9s
655:	learn: 0.4096650	total: 45.5s	remaining: 23.8s
656:	learn: 0.4096539	total: 45.5s	remaining: 23.8s
657:	learn: 0.4096472	total: 45.6s	remaining: 23.7s
658:	learn: 0.4096158	total: 45.7s	remaining: 23.6s
659:	learn: 0.4096027	total: 45.7s	remaining: 23.6s
660:	learn: 0.4095802	total: 45.8s	remaining: 23.5s
661:	learn: 0.409557

...

280:	learn: 0.4171768	total: 24s	remaining: 1m 1s
281:	learn: 0.4171573	total: 24.1s	remaining: 1m 1s
282:	learn: 0.4171296	total: 24.2s	remaining: 1m 1s
283:	learn: 0.4170898	total: 24.3s	remaining: 1m 1s
284:	learn: 0.4170580	total: 24.4s	remaining: 1m 1s
285:	learn: 0.4170174	total: 24.5s	remaining: 1m 1s
286:	learn: 0.4169910	total: 24.6s	remaining: 1m 1s
287:	learn: 0.4169571	total: 24.7s	remaining: 1m
288:	learn: 0.4169110	total: 24.7s	remaining: 1m
289:	learn: 0.4168718	total: 24.8s	remaining: 1m
290:	learn: 0.4168408	total: 24.9s	remaining: 1m
291:	learn: 0.4168082	total: 25s	remaining: 1m
292:	learn: 0.4167745	total: 25.1s	remaining: 1m
293:	learn: 0.4167022	total: 25.2s	remaining: 1m
294:	learn: 0.4166783	total: 25.2s	remaining: 1m
295:	learn: 0.4166327	total: 25.3s	remaining: 1m
296:	learn: 0.4166042	total: 25.4s	remaining: 1m
297:	learn: 0.4165824	total: 25.5s	remaining: 1m
298:	learn: 0.4165498	total: 25.6s	remaining: 60s
299:	learn: 0.4165088	total: 25.7s	remaining: 59.9s

441:	learn: 0.4122756	total: 37.7s	remaining: 47.6s
442:	learn: 0.4122429	total: 37.8s	remaining: 47.5s
443:	learn: 0.4122242	total: 37.9s	remaining: 47.4s
444:	learn: 0.4121963	total: 38s	remaining: 47.3s
445:	learn: 0.4121714	total: 38s	remaining: 47.3s
446:	learn: 0.4121471	total: 38.1s	remaining: 47.2s
447:	learn: 0.4121229	total: 38.2s	remaining: 47.1s
448:	learn: 0.4121119	total: 38.3s	remaining: 47s
449:	learn: 0.4120741	total: 38.4s	remaining: 46.9s
450:	learn: 0.4120496	total: 38.5s	remaining: 46.8s
451:	learn: 0.4120382	total: 38.5s	remaining: 46.7s
452:	learn: 0.4120122	total: 38.6s	remaining: 46.6s
453:	learn: 0.4119878	total: 38.7s	remaining: 46.5s
454:	learn: 0.4119553	total: 38.8s	remaining: 46.4s
455:	learn: 0.4119491	total: 38.8s	remaining: 46.3s
456:	learn: 0.4119319	total: 38.9s	remaining: 46.2s
457:	learn: 0.4119189	total: 39s	remaining: 46.1s
458:	learn: 0.4118626	total: 39.1s	remaining: 46.1s
459:	learn: 0.4118428	total: 39.2s	remaining: 46s
460:	learn: 0.4118206	

601:	learn: 0.4088594	total: 51s	remaining: 33.7s
602:	learn: 0.4088418	total: 51s	remaining: 33.6s
603:	learn: 0.4088281	total: 51.1s	remaining: 33.5s
604:	learn: 0.4088171	total: 51.2s	remaining: 33.4s
605:	learn: 0.4088004	total: 51.3s	remaining: 33.3s
606:	learn: 0.4087714	total: 51.3s	remaining: 33.2s
607:	learn: 0.4087528	total: 51.4s	remaining: 33.2s
608:	learn: 0.4087380	total: 51.5s	remaining: 33.1s
609:	learn: 0.4087189	total: 51.6s	remaining: 33s
610:	learn: 0.4087068	total: 51.6s	remaining: 32.9s
611:	learn: 0.4086913	total: 51.7s	remaining: 32.8s
612:	learn: 0.4086744	total: 51.8s	remaining: 32.7s
613:	learn: 0.4086614	total: 51.9s	remaining: 32.6s
614:	learn: 0.4086524	total: 52s	remaining: 32.5s
615:	learn: 0.4086192	total: 52.1s	remaining: 32.5s
616:	learn: 0.4085950	total: 52.1s	remaining: 32.4s
617:	learn: 0.4085855	total: 52.2s	remaining: 32.3s
618:	learn: 0.4085673	total: 52.3s	remaining: 32.2s
619:	learn: 0.4085549	total: 52.4s	remaining: 32.1s
620:	learn: 0.408526

763:	learn: 0.4061337	total: 1m 4s	remaining: 20s
764:	learn: 0.4061148	total: 1m 4s	remaining: 19.9s
765:	learn: 0.4061022	total: 1m 4s	remaining: 19.8s
766:	learn: 0.4060852	total: 1m 4s	remaining: 19.7s
767:	learn: 0.4060730	total: 1m 4s	remaining: 19.6s
768:	learn: 0.4060558	total: 1m 5s	remaining: 19.5s
769:	learn: 0.4060398	total: 1m 5s	remaining: 19.5s
770:	learn: 0.4060331	total: 1m 5s	remaining: 19.4s
771:	learn: 0.4060133	total: 1m 5s	remaining: 19.3s
772:	learn: 0.4059875	total: 1m 5s	remaining: 19.2s
773:	learn: 0.4059621	total: 1m 5s	remaining: 19.1s
774:	learn: 0.4059207	total: 1m 5s	remaining: 19s
775:	learn: 0.4059050	total: 1m 5s	remaining: 19s
776:	learn: 0.4058965	total: 1m 5s	remaining: 18.9s
777:	learn: 0.4058846	total: 1m 5s	remaining: 18.8s
778:	learn: 0.4058602	total: 1m 5s	remaining: 18.7s
779:	learn: 0.4058483	total: 1m 6s	remaining: 18.6s
780:	learn: 0.4058400	total: 1m 6s	remaining: 18.5s
781:	learn: 0.4058096	total: 1m 6s	remaining: 18.4s
782:	learn: 0.4057

921:	learn: 0.4036029	total: 1m 18s	remaining: 6.62s
922:	learn: 0.4035811	total: 1m 18s	remaining: 6.54s
923:	learn: 0.4035633	total: 1m 18s	remaining: 6.45s
924:	learn: 0.4035567	total: 1m 18s	remaining: 6.37s
925:	learn: 0.4035471	total: 1m 18s	remaining: 6.28s
926:	learn: 0.4035347	total: 1m 18s	remaining: 6.2s
927:	learn: 0.4035165	total: 1m 18s	remaining: 6.11s
928:	learn: 0.4035028	total: 1m 18s	remaining: 6.03s
929:	learn: 0.4034959	total: 1m 18s	remaining: 5.94s
930:	learn: 0.4034732	total: 1m 19s	remaining: 5.86s
931:	learn: 0.4034647	total: 1m 19s	remaining: 5.77s
932:	learn: 0.4034567	total: 1m 19s	remaining: 5.69s
933:	learn: 0.4034368	total: 1m 19s	remaining: 5.6s
934:	learn: 0.4034068	total: 1m 19s	remaining: 5.52s
935:	learn: 0.4033978	total: 1m 19s	remaining: 5.43s
936:	learn: 0.4033856	total: 1m 19s	remaining: 5.34s
937:	learn: 0.4033794	total: 1m 19s	remaining: 5.26s
938:	learn: 0.4033754	total: 1m 19s	remaining: 5.17s
939:	learn: 0.4033540	total: 1m 19s	remaining: 5